<a href="https://colab.research.google.com/github/ratulb/pytorch/blob/main/mnist_in_pythor_from_scratch_cleaned_wip3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import random

# Define a transform to convert the data to tensor
to_tensor = transforms.ToTensor()

# Load the MNIST dataset

In [2]:
# Download the training and test datasets
train_dataset = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=to_tensor)
#test_dataset = torchvision.datasets.MNIST(root='./', train=False, download=True, transform=to_tensor)
train_ds, val_ds = random_split(train_dataset, [50000, 10000])
len(train_ds), len(val_ds)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16149996.52it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 485776.65it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4462366.35it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4545580.71it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



(50000, 10000)

In [3]:
batch_size = 4

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [4]:
def convert_to_list(dataset):
    images = []
    labels = []
    for img, label in dataset:
        # Convert the tensor image to a list
        img_list = img.squeeze().tolist()
        images.append(img_list)
        labels.append(label)
    return images, labels

In [6]:
random.seed(24)

def one_hot_encode(values, num_classes):
    """
    Convert a list of values into one-hot encoded format.
    Args:
        values (list of list): List containing the values to encode.
        num_classes (int): Number of classes for one-hot encoding.
    Returns:
        list of list: One-hot encoded representation of the input values.
    """
    one_hot_encoded = []
    for value in values:
        one_hot = [0] * num_classes
        one_hot[value[0]] = 1
        one_hot_encoded.append(one_hot)
    return one_hot_encoded

#values = [[1], [0], [2], [3]]
#num_classes = 4

#one_hot_encoded_values = one_hot_encode(values, num_classes)
#print(one_hot_encoded_values)


In [7]:
def forward_pass(input_batch, all_neurons_weights, biases):
    batch_output = []
    for inputs in input_batch:
        layer_output = []
        for weights, bias in zip(all_neurons_weights, biases):
            neuron_output = sum(x * w for x, w in zip(inputs, weights)) + bias
            layer_output.append(neuron_output)
        batch_output.append(layer_output)
    return batch_output

def calculate_errors(batch_output_of_forward_pass, target_batch):
    squared_errors = []
    residual_errors = []
    for output, target in zip(batch_output_of_forward_pass, target_batch):
        try:
            sample_squared_errors = [(o - t) ** 2 for o, t in zip(output, target)]
        except OverflowError as e:
            print("Overflow error.", e)
            sample_squared_errors = 0
        #sample_squared_errors = [(o - t) ** 2 for o, t in zip(output, target)]
        sample_residual_errors = [(o - t) for o, t in zip(output, target)]
        squared_errors.append(sample_squared_errors)
        residual_errors.append(sample_residual_errors)
    return squared_errors, residual_errors

def calculate_weight_and_bias_deltas(output_of_calculate_errors, input_batch, learning_rate):
    residual_errors = output_of_calculate_errors[1]
    weight_deltas = [[0 for _ in range(len(input_batch[0]))] for _ in range(len(residual_errors[0]))]
    bias_deltas = [0 for _ in range(len(residual_errors[0]))]

    for residual_error, inputs in zip(residual_errors, input_batch):
        for neuron_index in range(len(residual_error)):
            for feature_index in range(len(inputs)):
                weight_deltas[neuron_index][feature_index] += learning_rate * residual_error[neuron_index] * inputs[feature_index]
            bias_deltas[neuron_index] += learning_rate * residual_error[neuron_index]

    return weight_deltas, bias_deltas

def update_weights_biases(output_of_calculate_weight_and_bias_deltas, weights, biases):
    weight_deltas, bias_deltas = output_of_calculate_weight_and_bias_deltas
    for i in range(len(weights)):
        for j in range(len(weights[i])):
            weights[i][j] -= weight_deltas[i][j]
        biases[i] -= bias_deltas[i]
    return weights, biases

def train(num_epochs, learning_rate):
    num_features = 784
    num_neurons = 10

    weights = [[random.random() for _ in range(num_features)] for _ in range(num_neurons)]
    biases = [random.random() for _ in range(num_neurons)]

    for epoch in range(num_epochs):
        for input_batch, target_batch in train_loader:
            input_batch =input_batch.squeeze().tolist()
            input_batch = [[item for sublist in outer for item in sublist] for outer in input_batch]
            target_batch = target_batch.tolist()
            target_batch = [[l] for l in target_batch]
            target_batch = one_hot_encode(target_batch, 10)
            batch_output = forward_pass(input_batch, weights, biases)
            errors = calculate_errors(batch_output, target_batch)
            deltas = calculate_weight_and_bias_deltas(errors, input_batch, learning_rate)
            weights, biases = update_weights_biases(deltas, weights, biases)

        if epoch % 2 == 0:
            total_error = sum(sum(e) for e in errors[0])
            print(f"Epoch {epoch}, Error: {total_error}")

    return weights, biases

num_epochs = 1
learning_rate = 0.001


weights, biases = train(num_epochs, learning_rate)
print("Trained weights:", weights)
print("Trained biases:", biases)



Epoch 0, Error: 4.7618676589645075
Trained weights: [[0.7123429878269185, 0.8397997363118733, 0.18259188695451745, 0.9982826275179507, 0.19409547872374744, 0.6708867302659983, 0.09178661335919236, 0.7577369809057047, 0.15130025749194376, 0.7065646893037225, 0.7244493284983917, 0.7659427822583291, 0.4412144832861917, 0.8654148854656293, 0.9879089492200086, 0.11559376141555855, 0.5168944758821384, 0.9581608361848446, 0.7391495693118509, 0.4442073008556111, 0.30460098013102177, 0.953031226956317, 0.08511839574001756, 0.7605559472285853, 0.8432547570122004, 0.15770052880945562, 0.3268706199544055, 0.30535819436979417, 0.5420861441566243, 0.3669973735567701, 0.8936907129977585, 0.9246322156021657, 0.3160669144878086, 0.8943942202636546, 0.06565681410140564, 0.08622843098134715, 0.6408122117757479, 0.7659823325287374, 0.7649253652605918, 0.9280686961871193, 0.8022753642449296, 0.22240308648188137, 0.6845807191502971, 0.8748045045456109, 0.06296209367486333, 0.47302896073863315, 0.14458601452